In [264]:
import os
from pathlib import Path
from collections import namedtuple
import math
from functools import lru_cache
import re

FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day19.txt'

In [265]:
class Supply(namedtuple("Supply", ['ore', 'clay', 'obsidian', 'geode'])):
    def __add__(self, other):
        return Supply(*(s + o for s, o in zip(self, other)))

    def __sub__(self, other):
        return Supply(*(s - o for s, o in zip(self, other)))

    def __mul__(self, n):
        return Supply(*(s * n for s in self))
    
    def time_to_make(self, bot_count):
        '''
        Custom behavior here: ceiling division &
        Return None when dividing by zero!!
        '''        
        if all(need <= 0 for need in self):
            return 1
        try:
            return max([math.ceil(need/rate) + 1 for need, rate in zip(self, bot_count) if need > 0])
        except ZeroDivisionError as e:
            return None
    
    def __rmul__(self, n):
        return self * n
    
    def __gte__(self, other):
        return all(s >= o for s, o in zip(self, other))

# supply count
inventory = Supply(0,0,0,0)

# bot count
robots = Supply(1,0,0,0)

# bot costs
rules = (
    Supply(2, 0, 0, 0),
    Supply(3, 0, 0, 0),
    Supply(3, 8, 0, 0),
    Supply(3, 0, 12, 0)
)

rules = []
with open(FOLDER / in_file) as f:
    for line in f:
        n, *r = map(int, re.findall(r'\d+', line))
        rules.append((
            Supply(r[0], 0, 0, 0),
            Supply(r[1], 0, 0, 0),
            Supply(r[2], r[3], 0, 0),
            Supply(r[4], 0, r[5], 0)
        ))


In [242]:
@lru_cache(maxsize=None)
def best_run(inventory, robots, rules, time, run_until):
    '''Return the count of obsidian'''
    
    #print(f"T:{time} | I: {inventory} B: {robots}")

    time_needed = [(r - inventory).time_to_make(robots)  for r in rules]
    max_geode = inventory.geode

    # if we can't make any more robots, skip ahead to the desired time:
    if all(t is None or t + time > run_until for t in time_needed):
        new_inventory = inventory + robots * (run_until - time)
        return new_inventory.geode

    # if we can buy a geode bot do it!
    if all(inv <= 0 for inv in rules[3] - inventory):
        future_bots = robots + (n == 3 for n in range(4))
        new_inventory = (inventory - rules[3]) + robots 
        return best_run(new_inventory, future_bots, rules, time + 1, run_until)
    
    for bot, t in enumerate(time_needed):
        current = 0
            
        # don't spend a lot of time going down
        # ore-hoarding branches. 
        if t is None:
            continue
        else:
            # new bot state
            future_bots = robots + (n == bot for n in range(4))

            # subtract cost 
            new_inventory = (inventory - rules[bot]) + robots * t

            if time + t <= run_until:
                current = best_run(new_inventory, future_bots, rules, time + t, run_until)
                
        if current > max_geode:
            max_geode = current
    
    return max_geode

runs = []
for i, rule in enumerate(rules, 1):
    best = best_run(inventory, robots, rule, time=0, run_until=24)
    runs.append(best)
    print(i, best)
            

1 0
2 0
3 1
4 5
5 3
6 2
7 0
8 1
9 14
10 0
11 1
12 0
13 5
14 6
15 0
16 0
17 1
18 0
19 15
20 6
21 0
22 5
23 0
24 0
25 6
26 4
27 1
28 0
29 8
30 0


In [244]:
print(runs)
sum([r * i for i, r in enumerate(runs, 1)])

[0, 0, 1, 5, 3, 2, 0, 1, 14, 0, 1, 0, 5, 6, 0, 0, 1, 0, 15, 6, 0, 5, 0, 0, 6, 4, 1, 0, 8, 0]


1389

In [266]:
@lru_cache(maxsize=None)
def best_run(inventory, robots, rules, time, run_until):
    '''Return the count of obsidian'''
    
    #print(f"T:{time} | I: {inventory} B: {robots}")

    time_needed = [(r - inventory).time_to_make(robots)  for r in rules]
    max_geode = inventory.geode

    # if we can't make any more robots, skip ahead to the desired time:
    if all(t is None or t + time > run_until for t in time_needed):
        new_inventory = inventory + robots * (run_until - time)
        return new_inventory.geode

    # if we can buy a geode bot do it!
    if all(inv <= 0 for inv in rules[3] - inventory):
        future_bots = robots + (n == 3 for n in range(4))
        new_inventory = (inventory - rules[3]) + robots 
        return best_run(new_inventory, future_bots, rules, time + 1, run_until)
    
    for bot, t in enumerate(time_needed):
        current = 0
        if t is None:
            continue
            
        # don't spend a lot of time going down
        # ore-hoarding branches. 
        if inventory.clay > 0 and bot==0:
            continue
                 
        else:
            # new bot state
            future_bots = robots + (n == bot for n in range(4))

            # subtract cost 
            new_inventory = (inventory - rules[bot]) + robots * t

            if time + t <= run_until:
                current = best_run(new_inventory, future_bots, rules, time + t, run_until)
                
        if current > max_geode:
            max_geode = current
    
    return max_geode

runs = []
for i, rule in enumerate(rules[:3], 1):
    best = best_run(inventory, robots, rule, time=0, run_until=32)
    runs.append(best)
    print(i, best)
                 

1 11
2 13
3 21


In [267]:
import math
math.prod(runs)

3003